In [23]:
import os
import datetime as dt
import pandas as pd
import numpy as np
from datetime import timedelta
import math
import plotly.graph_objects as go
from denari import NarcoAnalytics as narc, Montana as mn, TaxTools as tax

In [24]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
slash = '/'
path = os.getcwd()
export_loc = path + slash + "Exports"
costs = pd.read_csv(path + slash + "spending.csv", infer_datetime_format=True)
pack = pd.read_csv(path + slash + "packages.csv", infer_datetime_format=True)
#clients = pd.read_csv(path + slash + "clients.csv", infer_datetime_format=True)
sales = pd.read_csv(path + slash + "calex.csv", infer_datetime_format=True)

x = [costs,pack,sales]
for i in x:
    i = narc.set_dates(i)
    
c = costs.copy()
s = sales.copy()

c = narc.fill_dates(c)
s = narc.fill_dates(s)

c = narc.split_dates(c,format='period')
s = narc.split_dates(s,format='period')
c

,date,weekday,day,week,month,quarter,year,tax year,purchase,quantity,shop,cost,shipping,type,asset
0,2022-01-01,5,1,52,2022-01,2022Q1,2022,2021/2022,Analytics,1.0,DenariAnalytics,20.00,0.0,subscription,NaN
1,2022-01-01,5,1,52,2022-01,2022Q1,2022,2021/2022,Prime,1.0,Amazon,3.99,0.0,subscription,NaN
2,2022-01-02,6,2,52,2022-01,2022Q1,2022,2021/2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-01-03,0,3,1,2022-01,2022Q1,2022,2021/2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-01-04,1,4,1,2022-01,2022Q1,2022,2021/2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354,2022-11-29,1,29,48,2022-11,2022Q4,2022,2022/2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN
355,2022-11-30,2,30,48,2022-11,2022Q4,2022,2022/2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN
356,2022-12-01,3,1,48,2022-12,2022Q4,2022,2022/2023,Prime,1.0,Amazon,3.99,0.0,subscription,NaN
357,2022-12-01,3,1,48,2022-12,2022Q4,2022,2022/2023,Gym Rent,1.0,GYMBOX,1200.00,0.0,bill,NaN


In [26]:
df = c.copy()

In [27]:
df

,date,weekday,day,week,month,quarter,year,tax year,purchase,quantity,shop,cost,shipping,type,asset
0,2022-01-01,5,1,52,2022-01,2022Q1,2022,2021/2022,Analytics,1.0,DenariAnalytics,20.00,0.0,subscription,NaN
1,2022-01-01,5,1,52,2022-01,2022Q1,2022,2021/2022,Prime,1.0,Amazon,3.99,0.0,subscription,NaN
2,2022-01-02,6,2,52,2022-01,2022Q1,2022,2021/2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-01-03,0,3,1,2022-01,2022Q1,2022,2021/2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-01-04,1,4,1,2022-01,2022Q1,2022,2021/2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354,2022-11-29,1,29,48,2022-11,2022Q4,2022,2022/2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN
355,2022-11-30,2,30,48,2022-11,2022Q4,2022,2022/2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN
356,2022-12-01,3,1,48,2022-12,2022Q4,2022,2022/2023,Prime,1.0,Amazon,3.99,0.0,subscription,NaN
357,2022-12-01,3,1,48,2022-12,2022Q4,2022,2022/2023,Gym Rent,1.0,GYMBOX,1200.00,0.0,bill,NaN


In [28]:
df['year_int'] = df['date'].dt.year
start_year = 2022
year_cluster = 2
final_year = 2024

df

,date,weekday,day,week,month,quarter,year,tax year,purchase,quantity,shop,cost,shipping,type,asset,year_int
0,2022-01-01,5,1,52,2022-01,2022Q1,2022,2021/2022,Analytics,1.0,DenariAnalytics,20.00,0.0,subscription,NaN,2022
1,2022-01-01,5,1,52,2022-01,2022Q1,2022,2021/2022,Prime,1.0,Amazon,3.99,0.0,subscription,NaN,2022
2,2022-01-02,6,2,52,2022-01,2022Q1,2022,2021/2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022
3,2022-01-03,0,3,1,2022-01,2022Q1,2022,2021/2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022
4,2022-01-04,1,4,1,2022-01,2022Q1,2022,2021/2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354,2022-11-29,1,29,48,2022-11,2022Q4,2022,2022/2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022
355,2022-11-30,2,30,48,2022-11,2022Q4,2022,2022/2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022
356,2022-12-01,3,1,48,2022-12,2022Q4,2022,2022/2023,Prime,1.0,Amazon,3.99,0.0,subscription,NaN,2022
357,2022-12-01,3,1,48,2022-12,2022Q4,2022,2022/2023,Gym Rent,1.0,GYMBOX,1200.00,0.0,bill,NaN,2022
